In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
import re
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, TensorBoard
from keras import metrics
import json
from helper_functions import import_data, neuralnet_model, batch_generator, count_stars
from sklearn.metrics import confusion_matrix
from os import listdir
import seaborn as sn

In [5]:
x = 2
if x>=4: print(1)
else: print(0)

0


In [4]:
for path in listdir('jsons_full'):
    print(count_stars('jsons_full/' + path))
    

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [ ]:
listdir('jsons_full_test')[0]

In [ ]:
y_test[y_test == 0].shape

In [ ]:
y_test.shape

In [ ]:
for X_, y_ in batch_generator('jsons_full_test', 1000):
    X_test = X_
    y_test = y_
    break

In [ ]:
X_test.nbytes

In [ ]:
np.float64(1).nbytes * 80000 / 1000000 * 250

In [ ]:
X_test[0, -1] = 1
X_test[0, -1]

In [ ]:
batch_size = 2000
nb_epoch = 10
samples_per_epoch = 15

for x_, y_ in batch_generator('json_test', batch_size, samples_per_epoch):
    X_debug = x_
    y_debug = y_
    break

In [ ]:
y_debug

In [ ]:
autoencoder_model_created = False
model_path = 'models/basic_autoencoder1.h5'

if not autoencoder_model_created:
    model = neuralnet_model(X_test)

    batch_size = 2000
    nb_epoch = 10
    samples_per_epoch = 15

    # instantiate callbacks
    tensorboard = TensorBoard(log_dir='./autoencoder_logs', histogram_freq=2, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True)
    earlystopping = EarlyStopping(monitor='val_loss', patience=2)

    # try different number of epochs - 10 gives good performanace 
    """model.fit(X_train, X_train, epochs=10, batch_size=batch_size, verbose=1,
              validation_split=0.1, callbacks = [earlystopping, tensorboard])""" # cross val to estimate test error


    model.fit_generator(generator=batch_generator('jsons_full', batch_size, samples_per_epoch),
                        epochs=nb_epoch,
                        steps_per_epoch=samples_per_epoch)


    scores = model.evaluate(X_test, y_test)
    print('Test mse = {}'.format(scores[0]))

    y_predictions = model.predict(X_test)
    
    model.save(model_path)

else:
    model = load_model(model_path)
    scores = model.evaluate(X_test, X_test)
    print('Test mse = {}'.format(scores[0]))


In [ ]:
confusions = confusion_matrix(y_test, y_predictions < 0.5)

In [ ]:
confusions

In [ ]:
true_negative = confusions[0,0]
false_positive = confusions[0,1]
false_negative = confusions[1, 0]
true_positive = confusions[1, 1]

In [ ]:
print('Recall: {:0.2f}'.format(true_positive / (true_positive + false_negative)))
print('Precision: {:0.2f}'.format(true_positive / (true_positive + false_positive)))

In [ ]:
conf_df = pd.DataFrame([[true_positive, false_positive], [false_negative, true_negative]], \
                       ['predicted positive', 'predicted negative'], \
                       ['condition positive', 'condition negative'])
conf_df

In [ ]:
plt.figure(figsize=(15,10))
sn.set(font_scale=3)
sn.heatmap(conf_df, annot=True, annot_kws={'size': 26}, fmt='g')

In [ ]:
X_test_decoded[X_test_decoded < 0.5].shape

In [ ]:
X_test_decoded[X_test_decoded < 0.5].shape

In [ ]:
X_test_decoded[X_test_decoded < 0.5].shape

In [ ]:
X_test_decoded[X_test_decoded < 0.5].shape

In [ ]:
y_test.shape

In [ ]:
y_test[y_test == 1].shape

In [ ]:
X_test_decoded